# Set Up Matrix Input

In [ ]:
import numpy as np
import time

# Row and Col size of matrix being bulitiplied
row_size = 4
col_size = 4
row_block = 4

# Generate random input matrix
# A = np.random.rand(row_size,col_size)
# B = np.random.rand(col_size,row_size)
# A = np.full((row_size,col_size),1)
# B = np.full((col_size,row_size),-1)
# A = np.random.uniform(low=0.0625,high=1,size=(row_size,col_size))
# B = np.random.uniform(low=0.0625,high=1,size=(col_size,row_size))

pos_values = [2**(-i) for i in range(0,2)]
neg_values = [-2**(-i) for i in range(0,2)]
values = np.concatenate([neg_values,pos_values,[0]])
print(values)
A_val = np.random.choice(values,row_size*col_size)
A = A_val.reshape(row_size,col_size)
B_val = np.random.choice(values,row_size*col_size)
B = B_val.reshape(col_size,row_size)

# A = np.zeros((row_size,col_size))
# A[::2, :] = 1
# A[1::2,:] = -0.5
# B = np.zeros((col_size,row_size))
# B[::2, :] = -0.5
# B[1::2,:] = 0.25

py_out = np.zeros((row_size,row_size))
py_out = np.matmul(A,B)

# print(f"{A}\n{B}\n{py_out}")

# Converters Float to Fixed Point and Vice Versa

In [2]:
def float_to_fp(value, total_bits=16, frac_bits=8, output_format='hex'):
    # Step 1: Check if the value is negative
    if value < 0:
        is_negative = True
        value = -value  # Work with the positive equivalent for conversion
    else:
        is_negative = False

    # Step 2: Multiply the float by 2^frac_bits to scale it
    scaled_value = round(value * (2 ** frac_bits))
    
    # Step 3: Convert the scaled value to a binary string
    # Format the binary string to the required total bits length
    binary_value = bin(scaled_value)[2:].zfill(total_bits)

    # Step 4: Handle the negative case
    if is_negative:
        # Perform two's complement conversion for negative numbers
        # Subtract the binary value from the max value for two's complement
        max_value = (1 << total_bits)  # This is 2^total_bits
        binary_value = bin(max_value - scaled_value)[2:].zfill(total_bits)

    # Step 5: Convert the binary value to hexadecimal
    hex_value = hex(int(binary_value, 2))[2:].zfill(total_bits // 4)  # Convert to hex and remove '0x'

    # Step 6: Format output based on the `output_format` argument
    if output_format == 'binary':
        return binary_value
    elif output_format == 'hex':
        return hex_value
    elif output_format == 'both':
        return (f"{total_bits}'b{binary_value}", f"{total_bits}'h{hex_value}")
    else:
        raise ValueError("Invalid output format. Choose 'binary', 'hex', or 'both'.")
    
def fp_to_float(value,total_bits=16, frac_bits=8, input_format='hex'):
    if input_format == 'binary': fp_val = int(value,2)
    elif input_format == 'hex': fp_val = int(value,16)
    else: raise ValueError("Invalid input format. Must be 'binary' or 'hex'")
    # Calculate the scaling factor: 2^frac_width
    scaling_factor = 2 ** frac_bits
    # Handle two's complement if the sign bit is set (for negative numbers)
    sign_bit_mask = 1 << (total_bits - 1)  # Mask to isolate the sign bit
    
    # Check if the value is negative by examining the sign bit
    if fp_val & sign_bit_mask:  # If sign bit is 1, it's negative
        # Convert from two's complement
        fp_val = fp_val - (1 << total_bits)  # Apply the two's complement adjustment
    
    # Convert to float by dividing by the scaling factor
    float_value = fp_val / scaling_factor
    
    return float_value

# Example usage:
value = 64  # Example float number

num1 = float_to_fp(value)
inv_num1 = fp_to_float(str(num1))
print(num1,inv_num1)

4000 64.0


# Create File for RTL Testing

In [28]:
def wr_input(matrix,filename):
    # Matrix formatting
    new_row = row_block + 32//4 - 1 + row_block - 1 +1
    new_col = row_block
    sys_format = np.zeros((120,new_col))

    for i in range(row_size):
        for j in range(col_size):
            sys_format[i+j,j] = matrix[i,j]
    with open(filename, 'w') as f:
        for row in sys_format:
            data = [float_to_fp(i) for i in row]
            f.write(''.join(data) + '\n')
    # f = open(filename,'w')
    # f.close()
    # print(f'Formatted Systolic matrix:\n{sys_format}')

# print(f"Data North:\n{B}")
# wr_input(B,"..\\testing.txt")

In [29]:
import os

# Create blocks from the big matrix
rtl_out = np.zeros((row_size,row_size))

for i in range(row_size//4):
    for j in range(row_size//4):
        print(f"Data West:\n{A}")
        wr_input(A.transpose(),"..\\data_west.txt")
        print(f"Data North:\n{B}")
        wr_input(B,"..\\data_north.txt")
        command = f'cd .. & iverilog.exe -c .\\block_4x32.txt & vvp .\\a.out'
        os.system(command)

        out_temp = []
        with open('..\\out_val.txt', 'r') as f:
            # Iterate through each line in the file
            for line in f:
                temp_data = [line[i:i+4]for i in range(0, len(line.strip()), 4)]
                print(temp_data)
                line_data = [fp_to_float(line[i:i+4]) for i in range(0, len(line.strip()), 4)]
                out_temp.append(line_data)

        # out_temp = np.array(out_temp)
        # print(out_temp)
        print(i,j)
        rtl_out[4*i:4*(i+1),4*j:4*(j+1)] = out_temp
        err = py_out - rtl_out
        # print(err)


print(f"Python:\n{py_out}\nRTL:\n{rtl_out}\nError:\n{err}")
# print(err)
print(f'Maximum absolute error: {np.max(np.abs(err))}')

Data West:
[[ 0.   1.   0.5  1. ]
 [ 0.   0.  -1.  -0.5]
 [ 0.   1.  -0.5 -1. ]
 [ 0.5  0.   0.  -0.5]]
Data North:
[[-1.  -0.5  1.   1. ]
 [-0.5  0.5 -1.   1. ]
 [ 1.   0.  -0.5  0.5]
 [ 1.  -1.  -0.5 -1. ]]
['xxxx', 'xxxx', 'xxxx', 'xxxx']


ValueError: invalid literal for int() with base 16: 'xxxx'

[[ 0.   1.   0.5  1. ]
 [ 0.   0.  -1.  -0.5]
 [ 0.   1.  -0.5 -1. ]
 [ 0.5  0.   0.  -0.5]]
[[-1.  -0.5  1.   1. ]
 [-0.5  0.5 -1.   1. ]
 [ 1.   0.  -0.5  0.5]
 [ 1.  -1.  -0.5 -1. ]]
